In [1]:
import numpy as np
import pandas as pd

import random

from sklearn.datasets import make_classification

In [2]:
X, y = make_classification(n_samples=500, n_features=20, n_informative=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

In [3]:
#df = pd.read_csv(r'D:\Pythonizm\DS_Datasets\banknote+authentication.zip', header=None)
#df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'target']
#X, y = df.iloc[:,:4], df['target']

In [4]:
display(X.head())
print(X.shape)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19
0,-0.203674,-0.226479,1.226933,-0.826880,-0.014150,-0.582239,-0.602212,1.661259,-0.998385,0.766080,0.367366,-0.100154,-0.457096,0.468774,0.913585,-0.528639,-0.877983,0.301792,-0.803179,-0.678286
1,1.336657,1.221295,-0.895428,0.320217,0.202608,0.081888,0.184971,0.576451,1.707330,-1.278353,0.133958,-1.336569,-1.508556,0.452756,-0.095112,-1.374836,1.548639,-0.770819,-0.333038,0.751154
2,0.381734,-0.169852,0.124752,-0.500198,0.212906,-0.078315,-1.967325,1.090183,0.293538,-0.326673,1.636740,1.515445,-0.748096,-0.021245,-0.183301,-1.622804,-0.373368,0.249418,-0.019794,0.610911
3,0.630876,0.091168,-0.057964,2.792209,0.062873,1.061077,0.461272,0.039714,-0.450994,-0.772542,0.788204,-1.513831,-1.422877,-0.924357,-1.780626,0.695495,1.764914,0.293949,1.502734,1.357341
4,0.523961,-0.104968,-1.047158,-0.745168,0.180750,0.544902,-1.268137,1.255146,0.068372,0.331648,0.612791,1.711029,-0.497673,-0.168259,-0.025494,-0.715276,-0.399900,-1.746467,-0.217159,1.182324


(500, 20)


In [5]:
display(y.head())
print(y.shape)

0    0
1    0
2    0
3    1
4    0
dtype: int32

(500,)


In [6]:
class MyTreeClf():

    def __init__(self, max_depth = 5, min_samples_split = 2, max_leafs = 20):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0
        self.depth = 0


    def __repr__(self):
        return f'MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}'


    def _entropy(self, y):
        return -np.sum([(i/len(y))*np.log2(i/len(y)) if i>0 else 0 for i in y.value_counts(sort=False)])


    def get_best_split(self, X, y):
        X, y = X.copy(), y.copy()
        S0 = self._entropy(y)
        max_ig = -1e15
        for col in X:
            values = sorted(X[col].unique())
            dividers = [(values[i]+values[i+1])/2 for i in range(len(values)-1)]
            for divider in dividers:
                split_right = y[X[col] > divider]
                split_left = y[X[col] <= divider]
                S_right = (len(split_right)/len(y))*self._entropy(split_right)
                S_left = (len(split_left)/len(y))*self._entropy(split_left)
                ig = S0 - S_right - S_left
                if ig > max_ig:
                    col_name, split_value, max_ig = col, divider, ig
        return col_name, split_value, max_ig


    def is_leaf(self, X, y):
        if len(X) == 1:
            return True
        if len(X) < self.min_samples_split:
            return True
        if self._entropy(y) == 0:
            return True
        return False


    def _splitter(self, X, y, verbose, depth = 0):
        depth += 1
        split = self.get_best_split(X, y)
        self.model[depth][2**(depth-1) - self.rest[depth]] = list(split[:2])
        self.rest[depth] -= 1
        if verbose:
            print(split)
        left = X[split[0]] <= split[1]
        right = X[split[0]] > split[1]

        if not self.is_leaf(X[left], y[left]) and self.leafs_cnt < (self.max_leafs - 1) and depth < self.max_depth:
            if verbose:
                print('left', len(X[left]), depth+1)
            self._splitter(X[left], y[left], verbose, depth)
        else:
            if verbose:
                print(f'\nLeft Done | size: {len(X[left])} | entrope: {self._entropy(y[left])} | depth: {depth+1} | val: {np.sum(y[left])/len(y[left])}\n')
                self.leaf_sum += np.sum(y[left])/len(y[left])
            self.leafs_cnt += 1
            self.model[depth+1][2**(depth) - self.rest[depth+1]] = sum(y[left])/len(y[left])
            for i in range(self.max_depth - depth + 1):
                self.rest[depth + i + 1] -= 2**i

        if not self.is_leaf(X[right], y[right]) and self.leafs_cnt < (self.max_leafs - 1) and depth < self.max_depth:
            if verbose:
                print('right', len(X[right]), depth+1)
            self._splitter(X[right], y[right], verbose, depth)
        else:
            if verbose:
                print(f'Right Done | size: {len(X[right])} | entrope: {self._entropy(y[right])} | depth: {depth+1} | val: {np.sum(y[right])/len(y[right])} \n')
                self.leaf_sum += np.sum(y[right])/len(y[right])
            self.leafs_cnt += 1
            self.model[depth+1][2**(depth) - self.rest[depth+1]] = sum(y[right])/len(y[right])
            for i in range(self.max_depth - depth + 1):
                self.rest[depth + i + 1] -= 2**i


    def fit(self, X: pd.DataFrame, y: pd.Series, verbose = False):
        X, y = X.copy(), y.copy()
        self.leafs_cnt = 0
        self.depth = 0
        self.model = {}
        self.rest = {}
        self.leaf_sum = 0

        depth = 0
        while depth < self.max_depth + 1:
            depth += 1
            self.model[depth] = [[0, 0] for _ in range(2**(depth-1))]
            self.rest[depth] = 2**(depth-1)
       
        self._splitter(X, y, verbose)


    def print_tree(self):
        for layer in self.model:
            print(f'{layer} - {self.model[layer]}')
        print(self.leafs_cnt)
        print(round(self.leaf_sum, 6))

In [7]:
tree = MyTreeClf(max_depth=4, min_samples_split=5, max_leafs=20)
print(tree)

MyTreeClf class: max_depth=4, min_samples_split=5, max_leafs=20


In [8]:
tree.fit(X, y, True)

('col_15', 0.05279204081906824, 0.5000817440979197)
left 252 2
('col_5', 1.1697068696219022, 0.18239997909164657)
left 239 3
('col_4', -0.021314040925732358, 0.0815970686706685)
left 8 4
('col_8', -1.1019856361689269, 0.8112781244591328)

Left Done | size: 2 | entrope: -0.0 | depth: 5 | val: 0.0

Right Done | size: 6 | entrope: -0.0 | depth: 5 | val: 1.0 

right 231 4
('col_5', 0.09211433221500923, 0.04207443998708408)

Left Done | size: 158 | entrope: 0.05532848503795462 | depth: 5 | val: 0.006329113924050633

Right Done | size: 73 | entrope: 0.49867513632043436 | depth: 5 | val: 0.1095890410958904 

Right Done | size: 13 | entrope: -0.0 | depth: 3 | val: 1.0 

right 248 2
('col_5', 1.3607827903482992, 0.13613641900972567)
left 236 3
('col_15', 0.5097307543879097, 0.09218134809553963)
left 48 4
('col_4', -0.008237240619618339, 0.19879188180284735)

Left Done | size: 25 | entrope: 0.9988455359952018 | depth: 5 | val: 0.52

Right Done | size: 23 | entrope: 0.2580186686648155 | depth: 5 

In [9]:
tree.print_tree()

1 - [['col_15', 0.05279204081906824]]
2 - [['col_5', 1.1697068696219022], ['col_5', 1.3607827903482992]]
3 - [['col_4', -0.021314040925732358], 1.0, ['col_15', 0.5097307543879097], ['col_3', 1.1984896544656773]]
4 - [['col_8', -1.1019856361689269], ['col_5', 0.09211433221500923], [0, 0], [0, 0], ['col_4', -0.008237240619618339], ['col_10', 1.1428801257807422], 0.0, 1.0]
5 - [0.0, 1.0, 0.006329113924050633, 0.1095890410958904, [0, 0], [0, 0], [0, 0], [0, 0], 0.52, 0.9565217391304348, 1.0, 0.9, [0, 0], [0, 0], [0, 0], [0, 0]]
11
6.49244
